In [1]:
import numpy as np
import random
import math

In [2]:
class Particle:
  def __init__(self,position,velocity,fitness):
    self.position = position
    self.velocity  = velocity
    self.local_best_position = position
    self.fitness = fitness
  def update_local_best(self,local_best_position):
    self.local_best_position = local_best_position
  def update_fitness(self,fitness):
    self.fitness = fitness

In [3]:
# Define the modified fitness function to calculate MSE
def fitness_function(position):
    # Compute the squared error for each element
    squared_error = (position - 1) ** 2
    # Return the mean squared error (MSE)
    return np.mean(squared_error)

In [4]:
def calculate_global_best(fitness,swarm_best_fitness,position,swarm_global_best_position):
  if fitness < swarm_best_fitness:
          swarm_best_fitness = fitness
          swarm_global_best_position = position
  return swarm_global_best_position,swarm_best_fitness

In [8]:
def write_particle_data(filename, gene, particles, swarm_global_best_position, swarm_best_fitness):
  with open(filename, 'w') as f:
    f.write(f"Initial values for gene :{gene}\n")
    f.write("-" * 30 + "\n")
    for particle in particles:
      f.write(f"Particle {particles.index(particle)}\n")
      f.write(f"Position: {particle.position}\n")
      f.write(f"Velocity: {particle.velocity}\n")
      f.write(f"Local Best Position: {particle.local_best_position}\n")
      f.write(f"Fitness: {particle.fitness}\n")
      f.write("-" * 20 + "\n")
    f.write(f"Global Best Position: {swarm_global_best_position}\n")
    f.write(f"Global Best Fitness: {swarm_best_fitness}\n")
    f.write("-" * 50 + "\n")

In [6]:
def PSO(num_gene,num_particle,max_iteration,max_inertia,min_inertia,accel_factor):
    # Store all gbest for all gene
    gene_gbest_position =[]


    for gene in range(num_gene):
      particles = []    # Store all the particles
      swarm_global_best_position = None  # Initialization of global best
      swarm_best_fitness = float('inf')  # Initialization of fitness
      for population in range(num_particle):
        # Initialize position, velocity, and fitness for each particle
        position = np.random.uniform(0,1,num_gene+2)
        velocity = np.zeros(num_gene+2)
        fitness = fitness_function(position)
        # Update global best according
        swarm_global_best_position,swarm_best_fitness = calculate_global_best(fitness,swarm_best_fitness,position,swarm_global_best_position)
        particles.append(Particle(position,velocity,fitness))
        """
        print("Initial values for gene",gene)
        for particle in particles:
          print(f"Position: {particle.position}")
          print(f"Velocity: {particle.velocity}")
          print(f"Local Best Position: {particle.local_best_position}")
          print(f"Fitness: {particle.fitness}")
          print("-" * 20)  # Separator for better readability
        print(f"Global Best Position: {swarm_global_best_position}")
        print(f"Global Best Fitness: {swarm_best_fitness}")
        print("-" * 50)  # Separator for better readability
        """
      filename = f"particle_data_gene_{gene}.txt"
      write_particle_data(filename, gene, particles, swarm_global_best_position, swarm_best_fitness)
      for itr in range(max_iteration-1):
        w = max_inertia - (max_inertia - min_inertia) * itr / max_iteration
        for particle in particles:
          r1 = random.random()
          r2 = random.random()
          particle.velocity = (w * particle.velocity +
                                 accel_factor * r1 * (particle.local_best_position - particle.position) +
                                 accel_factor * r2 * (swarm_global_best_position - particle.position))

            # Update position
          particle.position += particle.velocity
          fitness = fitness_function(particle.position)
          if fitness < particle.fitness:
            particle.update_fitness(fitness)
            particle.update_local_best(particle.position)
          swarm_global_best_position,swarm_best_fitness = calculate_global_best(fitness,swarm_best_fitness,particle.position,swarm_global_best_position)
      gene_gbest_position.append(swarm_global_best_position);
    gbest_matrix = np.array(gene_gbest_position)
    # Extract the first N elements from each row to get an N x N matrix
    fij_matrix = gbest_matrix[:, :num_gene]
    return fij_matrix

In [12]:
if __name__ == '__main__':
  num_gene = 8
  num_regulator = 4
  num_particle = math.comb(num_gene,num_regulator)
  max_iteration = 10
  max_inertia = 0.9
  min_inertia = 0.4
  accel_factor = 2  # c1 = c2 = 2
  result_matrix = PSO(num_gene, num_particle, max_iteration, max_inertia, min_inertia, accel_factor)

  print("Final N x N matrix [fij]:")
  for row in result_matrix:
    for element in row:
      print("{:.10f}".format(element), end="    ")
    print()
    print()

Final N x N matrix [fij]:
2.2687294918    3.1445207630    3.5298066242    1.0147506200    1.2672155101    0.9848346325    1.5904262631    -0.1553476859    

1.7198516615    2.0428344879    0.5956008871    1.4254079856    1.7807956638    2.9344984794    1.4128426682    1.9870300444    

-0.0462659830    1.0022023313    2.0013557878    0.0737356219    0.8260641463    -0.4611917957    -0.0166441211    -0.2494526934    

0.5102056655    -0.4788005261    0.4601896578    1.1469029916    1.0058876801    0.1306849932    1.0311885122    1.9142635995    

1.3144008253    2.2337275873    -0.5889562573    0.8807236646    0.6241317143    -0.2383341345    0.6782459337    1.7535971647    

2.9041932310    0.8898965867    2.9051710649    0.6802549018    1.7341647954    1.1262326574    2.6800755522    1.4185566421    

0.2185561584    -0.3204479030    0.0921123024    0.7660745888    0.1129825050    -1.4024212514    0.2316211112    -0.4949409327    

4.3452705866    1.7371205415    2.8358347880    3.627